In [1]:
from random import randint, shuffle
import time
import sys
limit = 10 ** 6
sys.setrecursionlimit(limit)

# Двоичное дерево

In [2]:
class Node:
    
    def __init__(self, key, value=None, left=None, right=None):
        self.key = key
        self.value = [value]
        self.L = left
        self.R = right
        
    def __repr__(self):
        return f'Node {str(self.key)}'

    
class Tree():
    
    def __init__(self, root=None):
        self.root = root
        self.row = []
        
    def insert(self, key, value):
        if not self.root:
            self.root = Node(key, value)
        else:
            self.insert_to_node(key, value, self.root)
        
    def insert_to_node(self, key, value, node):
            if node.key == key:
                node.value.append(value)
                return
            if node.key > key:
                if node.L:
                    self.insert_to_node(key, value, node.L)
                else:
                    node.L = Node(key, value)
            else:
                if node.R:
                    self.insert_to_node(key, value, node.R)
                else:
                    node.R = Node(key, value)

    def insert_branch(self, node, branch, side):
        if side == 'R':
            while node.R:
                node = node.R
            node.R = branch
        if side == 'L':
            while node.L:
                node = node.L
            node.L = branch

    def search(self, key):
        if self.root:
            result = self.search_to_node(key, self.root)
            if isinstance(result, Node):
                return result
            else:
                return result[0]
        else:
            return None
        
    def search_with_parent(self, key):
        if self.root:
            result = self.search_to_node(key, self.root)
            if isinstance(result, Node):
                return (result, None)
            else:
                return result
        else:
            return None
                
    def search_to_node(self, key, node):
        if node.key == key:
            return node
        if node.key > key:
            if node.L:                
                result = self.search_to_node(key, node.L)
                if result:
                    if isinstance(result, Node):
                        return(result, node)
                    else:
                        return result
        else:
            if node.R:
                result = self.search_to_node(key, node.R)
                if result:
                    if isinstance(result, Node):
                        return(result, node)
                    else:
                        return result
    
    def remove(self, key):
            node, parent = self.search_with_parent(key)
            if (parent and not node.L and not node.R):
                if node == parent.L:
                    parent.L = None
                else:
                    parent.R = None
                return

            if (parent and node.L and not node.R):
                if node == parent.L:
                    parent.L = node.L
                else:
                    parent.R = node.L
                return

            if (parent and not node.L and node.R):
                if node == parent.L:
                    parent.L = node.R
                else:
                    parent.R = node.R
                return

            if parent:
                if node == parent.L:
                    if node.L:
                        parent.L = node.L
                        self.insert_branch(node.L, node.R, side='R')
                if node == parent.R:
                    if node.R:
                        parent.R = node.R
                        self.insert_branch(node.R, node.L, 'L')
            else:
                if (not node.L and not node.R):
                    self.root = None
                    return
                if node.L and not node.R:
                    self.root = node.L
                    return
                if not node.L and node.R:
                    self.root = node.R
                    return

                self.root = node.L
                self.insert_branch(node.L, node.R, side='R')
                
    def ride(self, order='incr'):   
        node = self.root
        result = []
        def ride_incr(node):
            if node == None:
                return
            ride_incr(node.L)
            result.append(node.key)
            ride_incr(node.R)
        def ride_decr(node):
            if node == None:
                return
            ride_decr(node.R)
            result.append(node.key)
            ride_decr(node.L)
        
        if order == 'incr':    
            ride_incr(node)
        if order == 'decr':
            ride_decr(node)
        return result


### тестирование двоичного дерева

In [13]:
N = 15000

In [14]:
ordered_arr = [x for x in range(N)]

In [15]:
shuffle_arr = [x for x in range(N)]

In [16]:
shuffle(shuffle_arr)

In [17]:
# добавление элементов в возрастающем  порядке
start = time.time()
oreder_tree = Tree()
for item in ordered_arr:
    oreder_tree.insert(item, randint(0, 10))
stop = time.time()
print('{:f}'.format(stop - start))


26.887304


In [18]:
# добавление элементов в случайном порядке
start = time.time()
shuffle_tree = Tree()
for item in shuffle_arr:
    shuffle_tree.insert(item, randint(0, 10))
stop = time.time()
print('{:f}'.format(stop - start))


0.105555


In [19]:
# поиск элементов в "перемешанном" дереве
start = time.time()
for item in ordered_arr:
    shuffle_tree.search(item)
stop = time.time()
print('{:f}'.format(stop - start))


0.077953


In [20]:
# поиск элементов в "возрастающем" дереве
start = time.time()
for item in ordered_arr:
    oreder_tree.search(item)
stop = time.time()
print('{:f}'.format(stop - start))

36.833683


In [21]:
# удаление элементов в "перемешанном" дереве
start = time.time()
for item in shuffle_arr:
    shuffle_tree.remove(item)
stop = time.time()
print('{:f}'.format(stop - start))

0.190746


In [22]:
# удаление элементов в "возрастающем" дереве
start = time.time()
for item in ordered_arr:
    oreder_tree.remove(item)
stop = time.time()
print('{:f}'.format(stop - start))

0.010512


### Вывод:

# ABL дерево

In [36]:
class ABLNode(Node):
    def __init__(self, key, value=None, left=None, right=None):
        super().__init__(key, value, left=None, right=None)
        
    @property
    def hight(self):        
        if self.L:
            left = self.L.hight
        else:
            left = -1
        if self.R:
            right = self.R.hight
        else:
            right = -1
        return max(left, right) + 1
    
        
class ABLTree(Tree):
    def __init__(self, root=None):
        super().__init__(root)
        
    def insert(self, key, value):
        if not self.root:
            self.root = ABLNode(key, value)
        else:
            self.insert_to_node(key, value, self.root)
            _, parent = self.search_with_parent(key)
            self.rebalance(parent)
        
    def insert_to_node(self, key, value, node):

            if node.key == key:
                node.value.append(value)
                return
            if node.key > key:
                if node.L:
                    self.insert_to_node(key, value, node.L)
                else:
                    node.L = ABLNode(key, value)
            else:
                if node.R:
                    self.insert_to_node(key, value, node.R)
                else:
                    node.R = ABLNode(key, value)

    def search(self, key):
        if self.root:
            result = self.search_to_node(key, self.root)
            if isinstance(result, ABLNode):
                return result
            else:
                return result[0]
        else:
            return None
        
    def search_with_parent(self, key):
        if self.root:
            result = self.search_to_node(key, self.root)
            if isinstance(result, ABLNode):
                return (result, None)
            else:
                return result
        else:
            return None
                
    def search_to_node(self, key, node):
        if node.key == key:
            return node
        if node.key > key:
            if node.L:                
                result = self.search_to_node(key, node.L)
                if result:
                    if isinstance(result, ABLNode):
                        return(result, node)
                    else:
                        return result
        else:
            if node.R:
                result = self.search_to_node(key, node.R)
                if result:
                    if isinstance(result, ABLNode):
                        return(result, node)
                    else:
                        return result
    
    def remove(self, key):
        node, parent = self.search_with_parent(key)
        if (parent and not node.L and not node.R):
            if node == parent.L:
                parent.L = None
            else:
                parent.R = None
            return
                
        if (parent and node.L and not node.R):
            if node == parent.L:
                parent.L = node.L
            else:
                parent.R = node.L
            self.rebalance(parent)
            return
         
        if (parent and not node.L and node.R):
            if node == parent.L:
                parent.L = node.R
            else:
                parent.R = node.R
            self.rebalance(parent)
            return
                
        if parent:
            if node == parent.L:
                if node.L:
                    parent.L = node.L
                    self.insert_branch(node.L, node.R, side='R')
            if node == parent.R:
                if node.R:
                    parent.R = node.R
                    self.insert_branch(node.R, node.L, 'L')
            self.rebalance(parent)
        else:
            if (not node.L and not node.R):
                self.root = None
                return
            if node.L and not node.R:
                self.root = node.L
                self.rebalance(node.L)
                return
            if not node.L and node.R:
                self.root = node.R
                self.rebalance(node.R)
                return
            
            self.root = node.L
            self.insert_branch(node.L, node.R, side='R')
            self.rebalance(self.root)
    
    def height(self, node):
        if node is None:
            return -1
        else:
            return node.hight
    
    def MPP(self, node):
        _, parent = self.search_with_parent(node.key)
        b = node.L
        c = b.R
        node.L = c
        b.R = node
        if parent:   
            if parent.L:
                if parent.L == node:
                    parent.L = b
            if parent.R:
                if parent.R == node:
                    parent.R = b
        else:
            self.root = b
    
    def MLP(self, node):
        _, parent = self.search_with_parent(node.key)
        b = node.R
        c = b.L
        node.R = c
        b.L = node
        if parent:   
            if parent.L:
                if parent.L == node:
                    parent.L = b
            
            if parent.R:
                if parent.R == node:
                    parent.R = b
        else:
            self.root = b
    
    def BPP(self, node):
        self.MLP(node.L)
        _, parent = self.search_with_parent(node.key)
        self.MPP(node)     

    def BLP(self, node):
        self.MPP(node.R)
        _, parent = self.search_with_parent(node.key)
        self.MLP(node)

    def find_min(self):
        node = self.root
        if node.L:
            return find_min(node.L)

    def rebalance(self, node):
        while node:
            diff = self.height(node.L) - self.height(node.R)
            left = -1
            right = -1
            if node.L:
                if node.L.L:
                    left = node.L.L.hight
                else:
                    left = 0
                if node.L.R:
                    right = node.L.R.hight
                else:
                    left = 0
            if (diff > 1) and (left > right):
                self.MPP(node)
            if (diff > 1) and (left < right):
                self.BPP(node)
            
            if node.R:
                if node.R.L:
                    left = node.R.L.hight
                else:
                    left = 0
                if node.R.R:
                    right = node.R.R.hight
                else:
                    left = 0
            if (diff < -1) and(left < right):
                self.MLP(node)
            if (diff < -1) and(left > right):
                self.BLP(node)
            _, node = self.search_with_parent(node.key)


### тестирование ABL дерева

In [44]:
N = 15000
ordered_arr = [x for x in range(N)]
shuffle_arr = [x for x in range(N)]
shuffle(shuffle_arr)

In [45]:
# добавление элементов в ABL дерево в возрастающем  порядке
start = time.time()
oreder_tree = ABLTree()
for item in ordered_arr:
    oreder_tree.insert(item, randint(0, 10))
stop = time.time()
print('{:f}'.format(stop - start))

122.535772


In [46]:
# добавление элементов в ABL дерево в случайном порядке
start = time.time()
shuffle_tree = ABLTree()
for item in shuffle_arr:
    shuffle_tree.insert(item, randint(0, 10))
stop = time.time()
print('{:f}'.format(stop - start))

132.599438


In [47]:
# поиск элементов в "перемешанном" ABL дереве
start = time.time()
for item in shuffle_arr:
    shuffle_tree.search(item)
stop = time.time()
print('{:f}'.format(stop - start))


0.065362


In [48]:
# поиск элементов в "возрастающем" ABL дереве
start = time.time()
for item in ordered_arr:
    oreder_tree.search(item)
stop = time.time()
print('{:f}'.format(stop - start))

0.060585


In [49]:
# удаление элементов в "перемешанном" ABL дереве
start = time.time()
for item in shuffle_arr:
    shuffle_tree.remove(item)
stop = time.time()
print('{:f}'.format(stop - start))

125.479369


In [50]:
# удаление элементов в "возрастающем" ABL дереве
start = time.time()
for item in shuffle_arr:
    oreder_tree.remove(item)
stop = time.time()
print('{:f}'.format(stop - start))

88.903517


### Вывод:

In [51]:
# создаем деревья для демонстрации 2-го варианта удаления.(порядок удаляемых ключей - тоже имеет значение)
# добавление элементов в ABL дерево в возрастающем  порядке
start = time.time()
oreder_tree = ABLTree()
for item in ordered_arr:
    oreder_tree.insert(item, randint(0, 10))
stop = time.time()
print('{:f}'.format(stop - start))

125.972688


In [52]:
# добавление элементов в ABL дерево в случайном порядке
start = time.time()
shuffle_tree = ABLTree()
for item in shuffle_arr:
    shuffle_tree.insert(item, randint(0, 10))
stop = time.time()
print('{:f}'.format(stop - start))

132.847955


In [53]:
# ключи для удаления беруться из ordered_arr, упорядоченного по возрастанию.

In [54]:
start = time.time()
for item in ordered_arr:
    shuffle_tree.remove(item)
stop = time.time()
print('{:f}'.format(stop - start))

49.273431


In [55]:
# удаление элементов в "возрастающем" ABL дереве
start = time.time()
for item in ordered_arr:
    oreder_tree.remove(item)
stop = time.time()
print('{:f}'.format(stop - start))

29.678093
